# Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

# Setting Image and Batch Size

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

# Loading the train, val, and test data

In [3]:
train = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("../../datasets/data/split/Zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-03 12:37:30.062540: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 12:37:30.062700: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Adding augmentation layer

In [4]:
# Using this method for M1 Mac CPU acces to run augmentation
with tf.device('/cpu:0'):
  data_augmentation = keras.Sequential(
  [
      layers.RandomFlip("horizontal"),
      layers.RandomRotation(0.1),
      layers.RandomZoom(0.2)
      # layers.RandomContrast(0.2),
      # layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
  ] 
  )

# Defining the model

In [6]:
inputs = keras.Input(shape=(100, 100, 3))
x = inputs
with tf.device('/cpu:0'):
  x = data_augmentation(x)
x = layers.Rescaling(1./255)(x)
x = data_augmentation(x)
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 128)       7385

# Adding Callbacks

In [7]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath="../../models/zenodo/m4_Augmentation.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

# Running the model

In [8]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-03 12:38:53.864961: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 12:38:53.865061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 3.4005 - accuracy: 0.2173

2022-08-03 12:38:58.790275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 6s 48ms/step - loss: 3.4005 - accuracy: 0.2173 - val_loss: 1.6527 - val_accuracy: 0.3736
Epoch 2/40
105/105 [==============================] - 5s 48ms/step - loss: 1.7816 - accuracy: 0.3054 - val_loss: 1.5224 - val_accuracy: 0.4292
Epoch 3/40
105/105 [==============================] - 5s 47ms/step - loss: 1.4834 - accuracy: 0.4768 - val_loss: 0.8691 - val_accuracy: 0.7083
Epoch 4/40
105/105 [==============================] - 5s 44ms/step - loss: 1.1462 - accuracy: 0.6051 - val_loss: 0.6954 - val_accuracy: 0.7625
Epoch 5/40
105/105 [==============================] - 5s 44ms/step - loss: 0.7780 - accuracy: 0.7003 - val_loss: 0.6547 - val_accuracy: 0.7681
Epoch 6/40
105/105 [==============================] - 5s 44ms/step - loss: 0.5948 - accuracy: 0.7774 - val_loss: 0.4726 - val_accuracy: 0.8236
Epoch 7/40
105/105 [==============================] - 5s 43ms/step - loss: 0.4684 - accuracy: 0.8310 - val_loss: 0.7997 - val_accuracy: 0.7389
Epoch 8/40

# Evaluating the model

In [9]:

model.evaluate(test)

load_model = keras.models.load_model("../../models/zenodo/m4_Augmentation.h5")
load_model.evaluate(test)

 8/23 [=========>....................] - ETA: 0s - loss: 0.4516 - accuracy: 0.9375

2022-08-03 12:42:04.544733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 17ms/step - loss: 0.4221 - accuracy: 0.9472


[0.4221227169036865, 0.9472222328186035]